# 1º Notebook - Dados Brutos

Este notebook serve para puxar os dados brutos. O primeiro dataset que será puxado é a composição atual do Índice Ibovespa (01/04/2025). Este dataset servirá para, além de mostrar os ativos na composição, indicar quais são seus respectivos pesos na carteira.

Após puxar a composição do Ibovespa, será preciso fazer algumas avaliações que seriam importantes para uma análise: tamanho da empresa, quanto tempo a empresa foi criada, o tempo que está na bolsa. Essas informações estão em alguns sites de análise de investimento, assim sendo, devemos utilizar o *web scraping* para puxar estes dados. Portanto, ao final do *scraping*, vamos juntar estes dois datasets em um só, sendo a carteira do Ibovespa com informações adicionais.

Além disso, é preciso puxar os dados de cotação dos ativos. Com a biblioteca *yahoofinance* ou *yfinance*, podemos puxar a cotação de maior período dos ativos que são listados no Ibovespa.

In [1]:
# imports
import yfinance as yf
import pandas as pd
import numpy as np
import sys

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configurando o pandas para o máximo de colunas ao mostrar dataframes
pd.set_option('display.max_columns', None)

## 1.1 Importação dos Dados

**Fonte dos Dados**

Abaixo baixamos um arquivo csv que contem a composição da Carteira do Ibovespa atualizada (01/04/2025).

[Este link](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm) leva diretamente ao arquivo. 

In [3]:
# Importando os dados
df_comp_ibov = pd.read_csv('data/composicao_ibovespa.csv', encoding='ISO-8859-1', sep=';', thousands='.', decimal=',')

# Ordenando pela ordem de participação
df_comp_ibov = df_comp_ibov.sort_values(by="Part. (%)", ascending=False)

In [4]:
# Visualizando o arquivo
display(df_comp_ibov)

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
81,VALE3,VALE,ON NM,4270903023,11.595
45,ITUB4,ITAUUNIBANCO,PN EJ N1,5272192664,7.925
59,PETR4,PETROBRAS,PN N2,4431132660,7.884
58,PETR3,PETROBRAS,ON N2,2255782178,4.409
28,ELET3,ELETROBRAS,ON N1,1977170723,3.862
...,...,...,...,...,...
62,PETZ3,PETZ,ON NM,293689428,0.058
26,CVCB3,CVC BRASIL,ON NM,525591097,0.053
5,AZUL4,AZUL,PN N2,326872005,0.051
50,LWSA3,LWSA,ON NM,389635155,0.050


In [5]:
# Verificando a composição do Ibovespa
df_comp_ibov['Part. (%)'].sum().round(2)

100.0

## 1.2 Web Scraping

Seria interessante analisar a carteira do IBOV não apenas com ações destacadas, mas sim de qual setor, segmento elas pertencem, qual o tamanho de mercado de cada uma, Anos de IPO. Estas informações vamos pegar no site [Investidor10](https://investidor10.com.br/acoes).

In [6]:
# lista de açõs para o web scraping
lista_ativos = df_comp_ibov['Código'].to_list()

print(f"Quantidade ativos: {len(lista_ativos)}")
print(lista_ativos)

Quantidade ativos: 87
['VALE3', 'ITUB4', 'PETR4', 'PETR3', 'ELET3', 'BBAS3', 'SBSP3', 'B3SA3', 'BBDC4', 'ABEV3', 'WEGE3', 'ITSA4', 'EMBR3', 'JBSS3', 'BPAC11', 'EQTL3', 'SUZB3', 'RENT3', 'RDOR3', 'PRIO3', 'BBSE3', 'RADL3', 'ENEV3', 'GGBR4', 'RAIL3', 'CMIG4', 'VIVT3', 'UGPA3', 'VBBR3', 'TOTS3', 'CPLE6', 'BBDC3', 'BRFS3', 'TIMS3', 'KLBN11', 'ENGI11', 'LREN3', 'ELET6', 'CCRO3', 'STBP3', 'BRAV3', 'HAPV3', 'CMIN3', 'ASAI3', 'EGIE3', 'SANB11', 'ALOS3', 'ISAE4', 'NTCO3', 'CSAN3', 'CXSE3', 'TAEE11', 'PSSA3', 'HYPE3', 'CPFE3', 'MULT3', 'CSNA3', 'CYRE3', 'GOAU4', 'MRFG3', 'FLRY3', 'RECV3', 'BRAP4', 'IRBR3', 'POMO4', 'IGTI11', 'COGN3', 'CRFB3', 'SLCE3', 'MGLU3', 'AZZA3', 'YDUQ3', 'BRKM5', 'USIM5', 'SMTO3', 'VIVA3', 'AURE3', 'RAIZ4', 'VAMO3', 'MRVE3', 'BEEF3', 'PCAR3', 'PETZ3', 'CVCB3', 'AZUL4', 'LWSA3', 'AMOB3']


In [7]:
# Imports das bibliotecas
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep

**Decisões a tomar**

O uso da biblioteca *Selenium* é necessário para clicar em alguns botões do site. Infelizmente, as bibliotecas *requests* e *BeautifulSoup* não conseguem realizar essas interações.


In [8]:
def botao_valores (navegador):
    while True:
        try:
            # aguardar o site carregar
            sleep(1)
            
            # Verificando se o botão das opcoes
            botao_valores = navegador.find_element('xpath', '/html/body/div[4]/div[2]/main/section/div/div[5]/div[3]/div[1]/div/ul/li[2]/span/span[1]/span/span[2]')

            # Clicando no botão para fechar a propaganda
            botao_valores.click()
            
            sleep(1) 
            
            #  Opção desejada de valores absolutos
            selecionar_opcao = navegador.find_element('xpath', '/html/body/span/span/span[2]/ul/li[2]')
            selecionar_opcao.click()
            
            
            # Sai do loop se o botão for encontrado e clicado
            break  
        except:
            print("Erro em localizar o botao")
            sleep(1)    

In [9]:
# Funcao para localizar as informações desejadas (o quandrante pode ficar mudando)
def localiza_info (navegador, info):
    nro_div = 8
    while True:       
        try:
            xpath_localiza = f'//*[@id="table-indicators-company"]/div[{nro_div}]/a/span[1]'
            localiza = navegador.find_element('xpath', xpath_localiza).text

            if localiza == info:
                xpath = f'//*[@id="table-indicators-company"]/div[{nro_div}]/a/span[2]'
                info = navegador.find_element('xpath', xpath).text
                break
            else:
                nro_div += 1 
        except:
            nro_div += 1 
            continue
    
    return info

In [10]:
def pegando_informacoes (ativo, navegador):
    
    # Constante para aguardar a proxima tarefa
    _TIME_TASK = 0.5
    
    
    # inicializando uma lista
    lista = []
    
    # Passando o primeiro index ativo
    lista.append(ativo)
    
    # Scraping das informações dos ativos
    Ano_Fundacao = navegador.find_element('xpath', '//*[@id="data_about"]/div[2]/div/div[1]/table/tbody/tr[5]/td[2]').text
    lista.append(Ano_Fundacao)
    
    sleep(_TIME_TASK)
    
    Ano_Ipo = navegador.find_element('xpath', '//*[@id="data_about"]/div[2]/div/div[1]/table/tbody/tr[3]/td[2]').text
    lista.append(Ano_Ipo)
    
    sleep(_TIME_TASK)
    
    Valor_Mercado = navegador.find_element('xpath', '//*[@id="table-indicators-company"]/div[1]/span[2]/div[2]').text
    lista.append(Valor_Mercado)
    
    sleep(_TIME_TASK)
    
    Setor = localiza_info (navegador, 'Setor')
    lista.append(Setor)

    sleep(_TIME_TASK)
    
    Segmento = localiza_info (navegador, 'Segmento')
    lista.append(Segmento)
    
    sleep(_TIME_TASK)
    
    return lista

In [11]:
def web_scraping (df, lista_ativos):
    count = 0
    for ativo in lista_ativos:
        count += 1
        print(f"Ativo: {ativo} - {count} de {len(lista_ativos)}")
        try:
            # Configurar o Chrome para rodar em modo headless
            options = Options()
            options.headless = True

            # Abrindo o navegador
            navegador = webdriver.Chrome(options=options)

            # ulr do status invest com o ativo passado
            url = f"https://investidor10.com.br/acoes/{ativo}"

            # Acessando a url
            sleep(0.5)
            navegador.get(url)

            # Maximizando a janela do navegador
            navegador.maximize_window()

            # Clicando no botao para apresentar valores
            botao_valores (navegador)

            sleep(0.5)
            # Lista com as informações
            lista = pegando_informacoes (ativo, navegador)

            df = pd.concat([df, pd.DataFrame([lista], columns=df.columns)], ignore_index=True)

            navegador.quit()
            
        except KeyboardInterrupt:
            print("\nWeb scraping interrompido pelo usuário.")
            sys.exit()
        except:
            print("error")
            continue
    return df

In [12]:
# Informações para o scraping
columns = ['Ativo', 'Ano Fundacao', 'Ano IPO', 'Valor Mercado', 'Setor', 'Segmento']


# Criando um dataframe para receber estas informações
df_scraping = pd.DataFrame(columns=columns)

In [13]:
%%time
# Fazendo web scraping para cada ativo
df_scraping = web_scraping (df_scraping, lista_ativos)

Ativo: VALE3 - 1 de 87
Ativo: ITUB4 - 2 de 87
Ativo: PETR4 - 3 de 87
Ativo: PETR3 - 4 de 87
Ativo: ELET3 - 5 de 87
Ativo: BBAS3 - 6 de 87
Ativo: SBSP3 - 7 de 87
Ativo: B3SA3 - 8 de 87
Ativo: BBDC4 - 9 de 87
Ativo: ABEV3 - 10 de 87
Ativo: WEGE3 - 11 de 87
Ativo: ITSA4 - 12 de 87
Ativo: EMBR3 - 13 de 87
Ativo: JBSS3 - 14 de 87
Ativo: BPAC11 - 15 de 87
Ativo: EQTL3 - 16 de 87
Ativo: SUZB3 - 17 de 87
Ativo: RENT3 - 18 de 87
Ativo: RDOR3 - 19 de 87
Ativo: PRIO3 - 20 de 87
Ativo: BBSE3 - 21 de 87
Ativo: RADL3 - 22 de 87
Ativo: ENEV3 - 23 de 87
Ativo: GGBR4 - 24 de 87
Ativo: RAIL3 - 25 de 87
Ativo: CMIG4 - 26 de 87
Ativo: VIVT3 - 27 de 87
Ativo: UGPA3 - 28 de 87
Ativo: VBBR3 - 29 de 87
Ativo: TOTS3 - 30 de 87
Ativo: CPLE6 - 31 de 87
Ativo: BBDC3 - 32 de 87
Ativo: BRFS3 - 33 de 87
Ativo: TIMS3 - 34 de 87
Ativo: KLBN11 - 35 de 87
Ativo: ENGI11 - 36 de 87
Ativo: LREN3 - 37 de 87
Ativo: ELET6 - 38 de 87
Ativo: CCRO3 - 39 de 87
Ativo: STBP3 - 40 de 87
Ativo: BRAV3 - 41 de 87
Ativo: HAPV3 - 42 de 8

In [14]:
# Visualizando as informações retiradas
display(df_scraping)

,Ativo,Ano Fundacao,Ano IPO,Valor Mercado,Setor,Segmento
0,VALE3,1943,1970,R$ 223.954.633.000,Materiais Básicos,Minerais Metálicos
1,ITUB4,1943,2002,R$ 313.335.024.000,Financeiro,Bancos
2,PETR4,1953,1977,R$ 426.601.191.000,"Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
3,PETR3,1953,1977,R$ 426.601.191.000,"Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
4,ELET3,1962,1971,R$ 98.986.984.000,Utilidade Pública,Energia Elétrica
...,...,...,...,...,...,...
82,PETZ3,2013,2020,R$ 1.776.090.000,Consumo não Cíclico,Produtos Diversos
83,CVCB3,2009,2013,R$ 1.035.414.000,Consumo Cíclico,Viagens e Turismo
84,AZUL4,2008,2017,R$ 986.716.000,Bens Industriais,Transporte Aéreo
85,LWSA3,1997,2020,R$ 1.502.906.000,Tecnologia da Informação,Programas e Serviços


In [15]:
# Preenchendo Valores vazios com NaN
df_scraping.replace('', np.nan, inplace=True)

# Preenchendo Valores NaN
df_scraping['Ano Fundacao'] = df_scraping['Ano Fundacao'].fillna(df_scraping['Ano IPO'])

In [16]:
# Realizando alguns tratamentos com string
df_scraping['Valor Mercado'] = df_scraping['Valor Mercado'].str.replace('R$ ', '', regex=False)
df_scraping['Valor Mercado'] = df_scraping['Valor Mercado'].str.replace('.', '', regex=False)
df_scraping['Valor Mercado'] = pd.to_numeric(df_scraping['Valor Mercado'])

In [17]:
# Realizando o tratamento valores inteiros
df_scraping['Ano Fundacao'] = pd.to_numeric(df_scraping['Ano Fundacao'])
df_scraping['Ano Fundacao'] = df_scraping['Ano Fundacao'].astype(int)

df_scraping['Ano IPO'] = pd.to_numeric(df_scraping['Ano IPO'])
df_scraping['Ano IPO'] = df_scraping['Ano IPO'].astype(int)

In [18]:
# Outras informações
df_scraping['Idade'] = 2025 - df_scraping['Ano Fundacao']
df_scraping['Tempo em Bolsa'] = 2025 - df_scraping['Ano IPO']

# Concatenando as participações
df_scraping['Peso Participacao'] = df_comp_ibov['Part. (%)'].tolist()

In [19]:
df_scraping

,Ativo,Ano Fundacao,Ano IPO,Valor Mercado,Setor,Segmento,Idade,Tempo em Bolsa,Peso Participacao
0,VALE3,1943,1970,223954633000,Materiais Básicos,Minerais Metálicos,82,55,11.595
1,ITUB4,1943,2002,313335024000,Financeiro,Bancos,82,23,7.925
2,PETR4,1953,1977,426601191000,"Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",72,48,7.884
3,PETR3,1953,1977,426601191000,"Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",72,48,4.409
4,ELET3,1962,1971,98986984000,Utilidade Pública,Energia Elétrica,63,54,3.862
...,...,...,...,...,...,...,...,...,...
82,PETZ3,2013,2020,1776090000,Consumo não Cíclico,Produtos Diversos,12,5,0.058
83,CVCB3,2009,2013,1035414000,Consumo Cíclico,Viagens e Turismo,16,12,0.053
84,AZUL4,2008,2017,986716000,Bens Industriais,Transporte Aéreo,17,8,0.051
85,LWSA3,1997,2020,1502906000,Tecnologia da Informação,Programas e Serviços,28,5,0.050


In [20]:
# Salvando o dataset para futuras aplicações
df_scraping.to_csv('data/scraping_ativos.csv', index=False, encoding='ISO-8859-1')

## 1.3 Cotação dos Ativos

Neste caso com 87 ativos no IBOV nem todos talvez tenham a cotação por igual, por exemplo, ITUB4 o IPO é de 2002, AZUL4 é de 2017, logo, vamos puxar o periodo total de cada ativo.

*Observação:* Depois em outra seção, certamente vamos filtrar os ativos por um periodo, claro, vamos verificar qual melhor periodo, pode ser que esta filtragem exclua alguns ativos e ver a correlação ou testar a hipotese se os percentuais continuam "iguais" ou se dá muita diferença. Mas agora vamos somente puxar os ativos.

In [21]:
# Se for a primeira vez dos ativos esta funcao serve para baixar os arquivos, visto que fica dando erro no yahoofinance em
# baixar alguns dados
def download_ativos (tickers, save=True):
    # Variaveis para guardar os ativos
    dicionario_ativos = {}


    # Baixar os dados históricos ()
    df_hist = pd.DataFrame()
    for acao in tickers:
        print(acao)
        dados = yf.download(acao, interval="1d", period='max', multi_level_index=False)
        
        # Adicionar coluna com o nome da ação
        dados["Ação"] = acao
        
        # Transformando coluna date no tipo 'Date'
        dados.index = pd.to_datetime(dados.index)
        
        # Colunas em PT
        dados = dados.rename(columns={'Open': 'Abertura', 'High': 'Máxima', 'Low': 'Mínima', 'Close': 'Fechamento'})        
        
        # Transformando em Dataframe
        df = pd.DataFrame(dados)
        
        # Resetando o indice e trazendo a coluna Data
        df = df.reset_index()
        
        # Arredondando as casas decimais
        df[['Abertura', 'Máxima', 'Mínima', 'Fechamento']] = df[['Abertura', 'Máxima', 'Mínima', 'Fechamento']].round(2)
        
        # Calculando o percentual
        df['Percentual'] = (df['Fechamento'].shift(1) / df['Fechamento']) - 1
        
        # Retirando a primeira linha que nao tinha valor
        df = df[1:]
        
        if save == True:
            df.to_csv(f'data/acoes/{acao}.csv', encoding='ISO-8859-1')

        dicionario_ativos[acao] = df
        
    return dicionario_ativos

In [22]:
# Os tickers do yfinance exige um .SA no final
tickers = df_scraping['Ativo'] + '.SA'

# Verificando a quantidade
print(len(tickers))

# Realizar o download
dicionario_ativos = download_ativos (tickers)

87
VALE3.SA
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


ITUB4.SA


[*********************100%***********************]  1 of 1 completed


PETR4.SA


...

AZUL4.SA


[*********************100%***********************]  1 of 1 completed


LWSA3.SA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

AMOB3.SA


In [23]:
# colocando tudo em df de cotacao
df_cotacao = pd.DataFrame()

for acao in tickers:
    df_cotacao = pd.concat([df_cotacao, dicionario_ativos[acao]], axis=0)
    
# Visualizando todas as cotações
display(df_cotacao)

,Date,Fechamento,Máxima,Mínima,Abertura,Volume,Ação,Percentual
1,2000-01-04,1.48,1.51,1.48,1.51,782400,VALE3.SA,0.027027
2,2000-01-05,1.48,1.48,1.47,1.47,1876800,VALE3.SA,0.000000
3,2000-01-06,1.48,1.52,1.48,1.48,792000,VALE3.SA,0.000000
4,2000-01-07,1.54,1.55,1.50,1.50,5347200,VALE3.SA,-0.038961
5,2000-01-10,1.66,1.66,1.63,1.63,2980800,VALE3.SA,-0.072289
...,...,...,...,...,...,...,...,...
72,2025-04-03,0.26,0.27,0.25,0.25,11801000,AMOB3.SA,0.000000
73,2025-04-04,0.25,0.26,0.24,0.26,11478800,AMOB3.SA,0.040000
74,2025-04-07,0.25,0.25,0.24,0.25,7155300,AMOB3.SA,0.000000
75,2025-04-08,0.24,0.26,0.23,0.25,17096600,AMOB3.SA,0.041667


In [25]:
# Realizando alguns tratamentos
df_cotacao = df_cotacao.drop(columns=['Máxima', 'Mínima', 'Abertura', 'Volume'])

df_cotacao = df_cotacao.rename({'Fechamento': 'Cotacao', 'Ação': 'Ativo'}, axis=1)

df_cotacao['Ativo'] = df_cotacao['Ativo'].str.replace('.SA', '')

df_cotacao['Percentual'] = df_cotacao['Percentual'].round(4)

In [27]:
# Salvando o dataset para futuras aplicações
df_cotacao.to_csv('data/cotacao_ativos.csv', index=False, encoding='ISO-8859-1')